In [86]:
import numpy
G = SymmetricGroup(6)
exponent = G.exponent()
o = G.order()
print("The exponent of the group is " + exponent.str())
# find the correct prime p for which we can translate the problem into Z/pZ for
def find_dixon_prime(exponent,G):
    candidate = next_prime(ceil(2*sqrt(G.order())))
    while (candidate % exponent != 1):
        candidate = next_prime(candidate+1)
    return candidate
    
p = find_dixon_prime(exponent,G)
print("the prime p is " + p.str())
F = FiniteField(p) # will work over field F = Z/pZ

# also for the conversion to finite fields require some z with ord_p(z) = e
# inefficient algorithm below (but is asymptotically irrelevant)
for candidate in range(1,p):
    if F(candidate).multiplicative_order() == exponent:
        z = F(candidate)
QZ = CyclotomicField(exponent)

def is_invariant(matrix, subspace):
    # Method 1: Using 'in'
    return all(matrix * v in subspace for v in subspace.basis())

def detailed_invariance_check(matrix, subspace):
    print("Subspace basis:", subspace.basis())
    print("Subspace dimension:", subspace.dimension())
    
    invariant_vectors = []
    non_invariant_vectors = []
    
    for v in subspace.basis():
        transformed = matrix * v
        print(f"Original vector: {v}")
        print(f"Transformed vector: {transformed}")
        
        # Check membership more explicitly
        try:
            is_in_subspace = transformed in subspace
            print(f"Is in subspace: {is_in_subspace}")
            
            if is_in_subspace:
                invariant_vectors.append(v)
            else:
                non_invariant_vectors.append(v)
        except Exception as e:
            print(f"Error checking membership: {e}")
    
    print("Invariant vectors:", invariant_vectors)
    print("Non-invariant vectors:", non_invariant_vectors)
    
    return len(non_invariant_vectors) == 0

def find_common_eigenvectors(matrices):
    """
    Find k common eigenvectors for k matrices over a finite field.
    
    Parameters:
    -----------
    matrices : list of k square matrices of the same dimension over a finite field
    
    Returns:
    --------
    list of k common eigenvectors
    """
    # Sanity checks
    if not matrices:
        raise ValueError("Empty list of matrices")
    
    k = matrices[0].nrows()
    F = matrices[0].base_ring()
    
    # Verify all matrices are the same size and over the same field
    if any(m.nrows() != k or m.base_ring() != F for m in matrices[1:]):
        raise ValueError("Matrices must be same size and over same field")
    
    # Start with eigenvectors of the first matrix
    common_eigenspaces = matrices[0].eigenspaces_right()
    
    # Iteratively find common eigenspaces
    for matrix in matrices[1:]:
        # Temporary list to store new common eigenspaces
        new_common_eigenspaces = []
        
        # Check each current eigenspace against the next matrix
        for (eigenvalue1, eigenspace1) in common_eigenspaces:
            # Find eigenspaces of the current matrix
            current_matrix_eigenspaces = matrix.eigenspaces_right()
            
            # Look for matching eigenspaces
            for (eigenvalue2, eigenspace2) in current_matrix_eigenspaces:
                # Check if eigenspaces intersect
                intersection = eigenspace1.intersection(eigenspace2)
                
                # If non-zero intersection, keep it
                if intersection.dimension() > 0:
                    new_common_eigenspaces.append((eigenvalue1, intersection))
        
        # Update common eigenspaces
        common_eigenspaces = new_common_eigenspaces
        
        # If no common eigenspaces remain, raise an error
        if not common_eigenspaces:
            raise ValueError("No common eigenvectors found")
    
    # Return basis vectors for the final common eigenspaces
    # Ensure we return exactly k linearly independent vectors
    common_vectors = []
    for (_, eigenspace) in common_eigenspaces:
        # Extend our list of vectors while maintaining linear independence
        for v in eigenspace.basis():
            # Try to add vector while maintaining linear independence
            test_space = (F^k).span(common_vectors + [v])
            if test_space.dimension() > len(common_vectors):
                common_vectors.append(v)
                
                # Stop if we have k vectors
                if len(common_vectors) == k:
                    return common_vectors
    
    # If we couldn't find k linearly independent vectors
    raise ValueError(f"Could not find {k} linearly independent common eigenvectors")

The exponent of the group is 60
the prime p is 61


In [87]:
# refer to Dixon's 1967 High Speed Computation of Group Characters paper for notation and implementation details

class DixonCharacterTable:
    def structure_constants(self,G):
        # method computing the structure constants of a group, i.e. a_rst is the number of pairs (x,y) in the conjugacy classes r and s such
        # that their product is in the conjugacy class t.

        # initialise empty dictionary
        result = {(i,j):([0]*self.classes_amount) for i in range(0,self.classes_amount) for j in range(0,self.classes_amount)}
        
        for i in range(0,self.classes_amount):
            for g in self.classes[i]:
                for j in range(0,self.classes_amount):
                    for h in self.classes[j]:
                        for k in range(0,self.classes_amount):
                            if (g*h == self.representatives[k]):
                                result[(i,j)][k] += 1
        return result
        
    def __init__(self, G):
        # initialise important data that will be used often during the computations

        # create a list of all conjugacy classes and their amount k, and fix a representative from each class
        self.classes = list(G.conjugacy_classes())
        self.classes_amount = len(self.classes)
        self.representatives = [k.representative() for k in self.classes]
        
        # compute the sizes of their classes, then in the notation of Dixon (1967) have h_j = class_sizes[j]
        self.class_sizes = [G.order() / G.centralizer(C.representative()).order() for C in self.classes]

        # compute the inverse class for each class, in Dixon (1967) have j' = inverse_class[j]
        # this is O(n^2) if we can hash conjugacy classes, but this isn't possible in sage
        # so it is O(nk^2)
        self.inverse_class = []
        for class_id in range(0,self.classes_amount):
            rep = self.representatives[class_id]
            inverse = rep.inverse()
            # compute which class contains the inverse
            for idx, cls in enumerate(self.classes,1):
                if inverse in cls:
                    self.inverse_class.append(idx-1)
                    break
        # compute the power classes for each class, for the powers 0,...,e-1. Required for conversion from finite field
        self.power_class = [ [] for _ in range(self.classes_amount)]
        for class_id in range(0, self.classes_amount):
            rep = self.representatives[class_id]
            self.power_class[class_id] = [];
            for power in range(0, exponent):
                element_to_check = rep^power
                # compute which class contains the power
                for idx, cls in enumerate(self.classes,1):
                    if element_to_check in cls:
                        self.power_class[class_id].append(idx-1)
                        break
        # initialise class matrices M_1, ..., M_k
        # computing all structure constants can be done in O(n^3)
        self.structure_constants = self.structure_constants(G)
        self.class_matrices = [matrix(F, self.classes_amount, self.classes_amount, lambda x,y: self.structure_constants[(i,x)][y]) for i in range(0,self.classes_amount)]
    def compute_normalised_vectors(self):
        # go through the eigenspaces and let consecutive matrices act on them until their dimensions are 1
        #v = self.class_matrices[0].eigenvectors_right()
        #for i in range(1,self.classes_amount):
            # compute one-dimensional common eigenspaces
            #print(v)
            #for j in range(0, len(v)):
            #    count = 0
            #    if (v[j][2] != 1):
            #        V = span(v[j-count][1])
            #        V_fixed = (self.class_matrices[i].base_ring()^self.classes_amount).span(V.basis())
            #        
            #        print("TEST")
            #        print(V)
            #        print(type(V))
            #        print(self.class_matrices[i])
            #        if (detailed_invariance_check(self.class_matrices[i],V)):
            #            print("invariant!")
            #        print("Matrix ring:", self.class_matrices[i].base_ring())
            #        print("Subspace ring:", V_fixed.base_ring())
            #        print("Matrix parent:", self.class_matrices[i].parent())
            #        print("Subspace parent:", V_fixed.parent())
            #        restricted_matrix = self.class_matrices[i].restrict(V_fixed)
            #        
        #
            #        
            #        evecs = restricted_matrix.eigenvectors_right()
            #        v += evecs
            #        v.pop(j-count)
            #        count += 1\
            # strip vectors and normalise them

         v = find_common_eigenvectors(self.class_matrices)
         for idx in range(0, len(v)):
             # invert with first element
             inverse = F.one()/(v[idx][0])
             v[idx] = inverse*v[idx]
         self.normalised_vecs = v
    # run after running compute_normalised_vecs
    def compute_dimensions(self):
        result = []
        for i in range(0, self.classes_amount):
            #k_i = |G|/d_i^2
            k = sum([(self.normalised_vecs[i][j]*self.normalised_vecs[i][self.inverse_class[j]])/F(self.class_sizes[j]) for j in range(0, self.classes_amount)])
            d_squared = o*F.one()/k
            # pick a square root of d_squared by Euler's criterion
            d = d_squared.sqrt(p)
            # we know that p > 2d, so
            if (p < 2*d):
                result.append(d)
            else:
                result.append(p-d)
        self.dimensions = result
    def compute_characters(self):
        # set the character table containing the finite field elements
        self.finite_field_chartable = matrix(F, self.classes_amount, self.classes_amount, lambda x,y: self.normalised_vecs[x][y]*self.dimensions[x]/F(self.class_sizes[y]))
        # convert them to complex numbers in Z[\zeta_e] with the formula from Dixon (1967)
        # implement the finite field to complex number conversion here
        zeta = QZ.gen()
        def convert_to_C(i,j):
            # converts theta(\chi_j^i), returns \chi_j^i
            result = 0
            for s in range(0,exponent):
            # compute m(s) and add it to result
                m = 0
                for n in range(0,exponent):
                    m += self.finite_field_chartable[i,self.power_class[j][n]]*z^(-s*n)
                result += int(m/exponent) * zeta^s
            return result   
        self.chartable = matrix(QZ, self.classes_amount, self.classes_amount, lambda x,y: convert_to_C(x,y))
                            

In [88]:
D = DixonCharacterTable(G)
print("The class matrices of G are ")
print(D.class_matrices)
print("The conjugacy class sizes of G are ")
print(D.class_sizes)
D.compute_normalised_vectors()
D.compute_dimensions()
print("The dimensions of the irreps of G are ")
print(D.dimensions)
D.compute_characters()
print("The chartable in the finite field Z/pZ is ")
print(D.finite_field_chartable)
print("The chartable in the cyclotomic extension Q(zeta_" + exponent.str() + ") is ")
print(D.chartable)

The class matrices of G are 
[[1 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 1], [ 0  1  0  0  0  0  0  0  0  0  0]
[15  0  2  0  3  0  0  0  0  0  0]
[ 0  6  0  3  0  3  0  2  0  0  0]
[ 0  0  1  0  0  0  0  0  2  0  0]
[ 0  8  0  0  0  1  0  4  0  0  0]
[ 0  0  8  0  3  0  6  0  4  5  0]
[ 0  0  0  0  0  2  0  0  0  0  3]
[ 0  0  4  0  9  0  0  0  1  5  0]
[ 0  0  0 12  0  3  0  1  0  0  6]
[ 0  0  0  0  0  6  0  8  0  0  6]
[ 0  0  0  0  0  0  9  0  8  5  0], [ 0  0  1  0  0  0  0  0  0  0  0]
[ 0  6  0  3  0  3  0  2  0  0  0]
[45  0  4  0  9  0  9  0  4  5  0]
[ 0  3  0  6  0  0  0  2  0  0  3]
[ 0  0  8  0  9  0  0  0  4  5  0]
[ 0 24  0  0  0 15  0 12  0  0 18]
[ 0  0  8  0  0  0  9  0  4  5  0]
[ 0 12  0 12  0  9  0 17  0  0  9]
[ 0  0  8  0  9  0  9  0 17 10  0]
[ 0 